In [1]:
import numpy as np

In [116]:
class ACO:
    def __init__(self, num_pop, graph):
        n = len(graph)

        self.population = [[0]] * num_pop # Each ant starts at city number 0
        self.dist = np.zeros(shape=(num_pop,)) # dist[ant] = distance made so far

        self.pheromon = np.random.normal(size=(n, n)) ** 2 # Some initialization, really

        self.adjacency_matrix = np.zeros(shape=(n, n), dtype=np.float32) # [i, j] = Euclidean distance between city i and j

        for i in range(n):
            self.adjacency_matrix[i] = np.apply_along_axis(np.linalg.norm, 1, graph[i] - graph)

    def main_loop(self, num_iter, alpha, beta, Q, rho):
        n = self.adjacency_matrix.shape[0]
        best_route = None
        min_dist = np.inf
        
        mu = 1 / self.adjacency_matrix
        mu = np.where(np.isinf(mu), 0, mu)

        for i in range(num_iter):
            self.population = [[0]] * len(self.population) # Each ant starts at city number 0
            self.dist = np.zeros_like(self.dist)
            delta_tau = np.zeros_like(self.pheromon) # To perform pheromon decay after loop ends
            for ant in range(len(self.population)):
                # Looping until ant hasn't visited all of the cities
                while len(self.population[ant]) != n:
                    # Getting the last city that ant have visited so far
                    current_city = self.population[ant][-1]

                    # Temporary variable to compute probabilities later
                    tmp = (self.pheromon[current_city] ** alpha) * (mu[current_city] ** beta)

                    # Probabilities itself
                    prob = tmp / tmp.sum() # nan's since it counts current city for wich mu = inf

                    # Cumulative probabilites to choose next city
                    cum_prob = np.cumsum(prob)

                    # Choosing next city while it hasn't been visited
                    next_city = np.searchsorted(cum_prob, np.random.rand())
                    while next_city in self.population[ant]:
                        next_city = np.searchsorted(cum_prob, np.random.rand())

                    self.population[ant].append(next_city)
                    self.dist[ant] += self.adjacency_matrix[current_city][next_city]

                    # Updating pheromon
                    self.pheromon[current_city][next_city] += Q / self.dist[ant]
                    delta_tau[current_city][next_city] += Q / self.dist[ant]

                if min_dist > self.dist[ant] and self.dist[ant] != 0:
                    min_dist = np.copy(self.dist[ant])
                    best_route = np.copy(self.population[ant])

            print("Iteration: %i\nMin distance: %.3f\nMin route %s" % (i + 1, min_dist, best_route)) 

            # Pheromone decay
            self.pheromon = (1 - rho) * self.pheromon + delta_tau

In [127]:
"""
Min distance: 39.234
Min route [0 3 2 1 4 5]
"""
graph = np.array([[0, 13], [0,26], [0,27], [10, 10], [5, 31], [5, 32]])
test = ACO(100, graph)
test.main_loop(num_iter=100, alpha=0.5, beta=0.5, rho=0.5, Q=50)

Iteration: 1
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 2
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 3
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 4
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 5
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 6
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 7
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 8
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 9
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 10
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 11
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 12
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 13
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 14
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 15
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 16
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iteration: 17
Min distance: 37.711
Min route [0 3 1 2 4 5]
Iterat

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20784\2454840158.py:20: RuntimeWarning: divide by zero encountered in divide
  mu = 1 / self.adjacency_matrix
